## 2-3. DB INSERT_yearly_album_chart

### 3사 앨범 순위 정보만 추출하여 DB에 Insert 하기

In [67]:
import pandas as pd
import pymysql

In [68]:
conn = pymysql.connect(host='127.0.0.1'
                , port=3306
                , user = 'root'
                , password='root1234'
                , db = 'stock_pj'
                , charset='utf8'
                ) 

In [69]:
cur = conn.cursor()
cur.execute('SHOW TABLES') # 연동 확인
cur.fetchall() # 테이블 이름 확인

(('agency_stock',),
 ('singer_agency',),
 ('top_rank_2018',),
 ('yearly_album_chart',))

### 1. DB에 'singer_agency' 정보를 가져와 3사 앨범 순위 정보만 추출하기

- singer_agency 테이블 불러오기

In [36]:
cur.execute('SELECT * FROM singer_agency;')
singer_agency =  cur.fetchall()
singer_agency_df = pd.DataFrame(singer_agency)
singer_agency_df.columns = ['singer', 'agency_name']
singer_agency_df.head()

,singer,agency_name
0,2PM,JYP
1,DAY6,JYP
2,ITZY (있지),JYP
3,NMIXX,JYP
4,Stray Kids (스트레이 키즈),JYP


* 연간 앨범 차트 데이터를 가져오기

In [37]:
yearly_album_chart_df = pd.read_csv('data/yearly_album_chart_with_date.csv')
yearly_album_chart_df.head()

,year,rank,title,singer,sales,release_date
0,2018,1,LOVE YOURSELF 結 ''Answer'',방탄소년단,2197808,2018.08.24
1,2018,2,LOVE YOURSELF 轉 ''Tear'',방탄소년단,1849537,2018.05.18
2,2018,3,DON''T MESS UP MY TEMPO - The 5th Album,EXO,1452030,2018.11.02
3,2018,4,0+1=1 (I PROMISE YOU),Wanna One (워너원),782562,2018.03.19
4,2018,5,1÷χ=1 (UNDIVIDED),Wanna One (워너원),641131,2018.06.04


- singer_agency의 singer와 비교하여 3사 앨범의 정보만 추출하기

In [38]:
total_yearly_album_df = pd.DataFrame()
for singer_1 in yearly_album_chart_df['singer']:
    for singer_2 in singer_agency_df['singer']:
        if singer_1 == singer_2:
            tmp_df = yearly_album_chart_df[yearly_album_chart_df['singer'] == singer_1]
            total_yearly_album_df = pd.concat([total_yearly_album_df, tmp_df])

In [39]:
# 중복 제거
total_yearly_album_df.drop_duplicates(inplace=True)

In [41]:
# 정렬
total_yearly_album_df.sort_values(['year', 'rank'], inplace=True)

In [42]:
# 인덱스 리셋
total_yearly_album_df.reset_index(drop=True, inplace=True)

In [44]:
# 확인
total_yearly_album_df.head()

,year,rank,title,singer,sales,release_date
0,2018,3,DON''T MESS UP MY TEMPO - The 5th Album,EXO,1452030,2018.11.02
1,2018,7,LOVE SHOT - The 5th Album Repackage,EXO,499849,2018.12.13
2,2018,10,What is Love?,TWICE,348797,2018.04.09
3,2018,13,NCT 2018 EMPATHY,NCT,330335,2018.03.14
4,2018,15,YES or YES,TWICE,325761,2018.11.05


In [40]:
# shape 확인: 총 127개 이다.
total_yearly_album_df.shape

(127, 6)

### 2. DB INSERT_yearly_album_chart

In [64]:
# insert
for i in range(len(total_yearly_album_df)):
    cur.execute(f"INSERT INTO yearly_album_chart (year, ranking, singer, title, sales, release_date) VALUES \
                ({total_yearly_album_df['year'][i]}, {total_yearly_album_df['rank'][i]}, '{total_yearly_album_df['singer'][i]}', '{total_yearly_album_df['title'][i]}', {total_yearly_album_df['sales'][i]}, '{total_yearly_album_df['release_date'][i]}');")

In [ ]:
cur.execute(f"UPDATE yearly_album_chart SET release_date = null WHERE release_date = 'nan';")

In [65]:
# db 반영
conn.commit()

In [72]:
conn.close()